## CESM2 - LARGE ENSEMBLE (LENS2)

- This Notebooks aims to compute the heat balance in the South Atlantic, defined by the difference of the meridional heat transport from the northern and southern boundaries and the total surface heat flux (area integral). Let's use the data from the control PI to get the trend of the system before the forcing. 

### Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np 
import dask
import cf_xarray
import intake
import cftime
import nc_time_axis
import intake_esm
import matplotlib.pyplot as plt
import pop_tools
from dask.distributed import Client, wait
from ncar_jobqueue import NCARCluster
import warnings, getpass, os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import cmocean
import dask

### Dask

In [ ]:
mem_per_worker = 30 # memory per worker in GB 
num_workers = 40 # number of workers
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB', walltime='2:00:00')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Load the data

In [ ]:
%%time
path = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-piControl.001/ocn/proc/tseries/month_1/b.e21.B1850.f09_g17.CMIP6-piControl.001.pop.h.SHF.*.nc'
ds_SHF = xr.open_mfdataset(path)
path = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-piControl.001/ocn/proc/tseries/month_1/b.e21.B1850.f09_g17.CMIP6-piControl.001.pop.h.N_HEAT.*.nc'
ds_N_HEAT = xr.open_mfdataset(path)
path = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-piControl.001/ocn/proc/tseries/month_1/b.e21.B1850.f09_g17.CMIP6-piControl.001.pop.h.TEND_TEMP.*.nc'
ds_TEND_TEMP = xr.open_mfdataset(path, chunks={'z_t': 1})
del path

### Get the variable dz

In [ ]:
dz=ds_TEND_TEMP.dz.isel(time=0)*0.01 # 0.01 to convert cm into m

### Import POP grid

In [ ]:
pop_grid = pop_tools.get_grid('POP_gx1v7')

# Temperature Tendency
ds_TEND_TEMP['TLONG'] = pop_grid.TLONG; ds_TEND_TEMP['TLAT'] = pop_grid.TLAT
ds_TEND_TEMP['ULONG'] = pop_grid.ULONG; ds_TEND_TEMP['ULAT'] = pop_grid.ULAT

# Total Surface Heat Flux
ds_SHF['TLONG'] = pop_grid.TLONG; ds_SHF['TLAT'] = pop_grid.TLAT
ds_SHF['ULONG'] = pop_grid.ULONG; ds_SHF['ULAT'] = pop_grid.ULAT

# Meridional Heat Transport
ds_N_HEAT['TLONG'] = pop_grid.TLONG; ds_N_HEAT['TLAT'] = pop_grid.TLAT
ds_N_HEAT['ULONG'] = pop_grid.ULONG; ds_N_HEAT['ULAT'] = pop_grid.ULAT

del pop_grid

### Calculate the difference in heat transport to latitudes closer to the equator and 34S
- We chose 34 instead of 34.5S because at 34S we are sure that there is no water leakage South Africa 

In [ ]:
%%time
ilan = 0 # northernmost latitude
ilas = -34 # southernmost latitude
ds_N_HEAT_diff=(ds_N_HEAT.N_HEAT.isel(transport_reg=1,lat_aux_grid=190)-ds_N_HEAT.N_HEAT.isel(transport_reg=1).sel(lat_aux_grid=ilas,method='nearest')).sum(dim='transport_comp').load()
del ds_N_HEAT
ds_N_HEAT_diff.plot()

### Cut and center the variable in the South Atlantic

#### 1- Temperature tendency

In [ ]:
# Cutting out and centering the variables in the South Atlantic
dask.config.set({"array.slicing.split_large_chunks": True})
ilon1, flon1, ilon2, flon2 = 307, 320, 0, 54 # longitude (initial (i), final (f)) 

ds_sa_TEND_TEMP=xr.combine_nested([[ds_TEND_TEMP.TEND_TEMP.where(
    (ds_TEND_TEMP.TEND_TEMP.TLAT >= ilas) & (ds_TEND_TEMP.TEND_TEMP.TLAT <= ilan), drop=True).isel(nlon = slice(ilon1,flon1)),
    ds_TEND_TEMP.TEND_TEMP.where(
    (ds_TEND_TEMP.TEND_TEMP.TLAT >= ilas) & (ds_TEND_TEMP.TEND_TEMP.TLAT <= ilan), drop=True).isel(nlon = slice(ilon2,flon2))]],
                                  concat_dim=['nlat','nlon'])   
ds_sa_TEND_TEMP.coords['nlon'] = (ds_sa_TEND_TEMP.coords['nlon'] + 180) % 360 - 180 
ds_sa_TEND_TEMP = ds_sa_TEND_TEMP.sortby(ds_sa_TEND_TEMP.nlon)

#### 2- Total Surface Heat Flux

In [ ]:
ds_sa_SHF=xr.combine_nested([[ds_SHF.SHF.where(
    (ds_SHF.SHF.TLAT >= ilas) & (ds_SHF.SHF.TLAT <= ilan), drop=True).isel(nlon = slice(ilon1,flon1)),
    ds_SHF.SHF.where(
    (ds_SHF.SHF.TLAT >= ilas) & (ds_SHF.SHF.TLAT <= ilan), drop=True).isel(nlon = slice(ilon2,flon2))]],
                                  concat_dim=['nlat','nlon'])   
ds_sa_SHF.coords['nlon'] = (ds_sa_SHF.coords['nlon'] + 180) % 360 - 180 
ds_sa_SHF = ds_sa_SHF.sortby(ds_sa_SHF.nlon)

#### 3- Area

In [ ]:
ds_sa_TAREA=xr.combine_nested([[ds_TEND_TEMP.TAREA.where(
    (ds_TEND_TEMP.TAREA.TLAT >= ilas) & (ds_TEND_TEMP.TAREA.TLAT <= ilan), drop=True).isel(nlon = slice(ilon1,flon1)),
    ds_TEND_TEMP.TAREA.where(
    (ds_TEND_TEMP.TAREA.TLAT >= ilas) & (ds_TEND_TEMP.TAREA.TLAT <= ilan), drop=True).isel(nlon = slice(ilon2,flon2))]],
                                  concat_dim=['nlat','nlon'])   
ds_sa_TAREA.coords['nlon'] = (ds_sa_TAREA.coords['nlon'] + 180) % 360 - 180 
ds_sa_TAREA = ds_sa_TAREA.sortby(ds_sa_TAREA.nlon)
del ds_SHF, ds_TEND_TEMP

### Mask the continent 

In [ ]:
mask_array = dict()
mask_ocean = 2 * np.ones((len(ds_sa_TEND_TEMP.coords['nlat']), # ocean
                          len(ds_sa_TEND_TEMP.coords['nlon']))
                        ) * np.isfinite(ds_sa_TEND_TEMP.isel(time=0))  
mask_land  = 1 * np.ones((len(ds_sa_TEND_TEMP.coords['nlat']), # continent
                          len(ds_sa_TEND_TEMP.coords['nlon']))
                        ) * np.isnan(ds_sa_TEND_TEMP.isel(time=0))  
mask_array = mask_ocean + mask_land
ds_sa_TAREA=ds_sa_TAREA.isel(time=0).where(mask_array != 1.)*1e-4 # 1e-4 to convert cm2 into m2

# ds_sa_TAREA=ds_sa_TAREA
del mask_array

### Integrate the SHF in the area

In [ ]:
%%time
ds_sa_SHF=ds_sa_SHF.compute()
ds_sa_SHF

In [ ]:
%%time
# actually compute ds_sa_TAREA here
ds_sa_TAREA=ds_sa_TAREA.compute()
ds_sa_TAREA

In [ ]:
ds_sa_SHF=ds_sa_TAREA.isel(z_t=0)*ds_sa_SHF*(1e-15) # PW (1e-15 to convert the units from W to PW) 
ds_sa_SHF=ds_sa_SHF.sum(dim=['nlat','nlon'],skipna=True) # PW

In [ ]:
# Test
ds_N_HEAT_diff.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','2001-01-01')).plot(label='MHTD')
ds_sa_SHF.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','2001-01-01')).plot(label='SHF')
plt.legend()
plt.grid()
plt.xlabel('Time [Years]')
plt.ylabel('PW')
#plt.savefig('Heat_balance.png',dpi=300,bbox_inches='tight')
plt.show()

### Compute the heat balance

In [ ]:
ds_SHF_N_HEAT_diff=ds_sa_SHF-ds_N_HEAT_diff # PW

In [ ]:
# Test
ds_SHF_N_HEAT_diff.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01','2001-01-01')).plot(label='HS',color='red')
plt.legend()
plt.grid()
plt.title(None)
plt.xlabel('Time [Years]')
plt.ylabel('PW')
#plt.savefig('Heat_storage.png',dpi=300,bbox_inches='tight')
plt.show()

<div class="alert alert-block alert-info">
Here it was necessary to do the difference and not the sum of the terms to get the heat balance. This is because the SHF convection is positive to the ocean. The balance is given by every heat flux entering from the surface (positive direction of the z-axis) is equal to every flux leaving from the meridional heat transport (positive direction of the y-axis). The meirdional heat transport has a positive y-axis direction, but the SHF has not a negative z-axis direction. 
</div>

### Compute the heat storage (HS) to compare with the heat storage due to the difference between the heat fluxes
- the vertical integral of the temperature tendency 

#### Equation: $$\rm{HS = \uprho_\uptheta~C_p~\int_{z_2}^{z_1}\uptheta_{(z)}'~dz},$$
##### where:
##### * HS is heat storage ($\rm{J~m^{-2}}$),
##### * $\uprho$ is the density of sea water,
##### * $\rm{C_p}$ is the specific heat of sea water,
##### * $\rm{z}$ is the depth limit on the calculation in meters,
##### * and $\uptheta$' is the potential temperature monthly anomaly (successor month minus previous month) at each depth in degress Kelvin or Celsius or, the temperature tendency. 

In [ ]:
%%time
warnings.simplefilter("ignore")
ds_HS_TEMP=ds_sa_TEND_TEMP*dz # 1- Multiply by dz. Unit: oC.s-1.m 
ds_HS_TEMP=ds_HS_TEMP*ds_sa_TAREA # 2- Multiply by the area. Unit: oC.s-1.m3
ds_HS_TEMP=ds_HS_TEMP.sum(dim=['z_t','nlon','nlat']) # 3- Integral in dz,dy,dx. Unit: oC.s-1.m3
ds_HS_TEMP=ds_HS_TEMP*1026 # 4- Multiply by the density of the sea water. Unit: oC.s-1.kg
ds_HS_TEMP=ds_HS_TEMP*3996 # 5- Multiply by the heat capacity of the sea water. Unit: W
ds_HS_TEMP=ds_HS_TEMP*1e-15 # 6- Get the variable in PW 

In [ ]:
ds_HS_TEMP

In [ ]:
%%time
ds_HS_TEMP=ds_HS_TEMP.load()

In [ ]:
# Test
ds_HS_TEMP.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01', '1850-01-01')).plot()
ds_SHF_N_HEAT_diff.resample(time='1Y', closed='left').mean('time').sel(time=slice('1101-01-01', '1850-01-01')).plot()

In [ ]:
ds_out_var = xr.merge([ds_HS_TEMP.rename('HS_TEND_TEMP'), # Heat Storage from temperature tendency
                       ds_SHF_N_HEAT_diff.rename('HS'), # Heat Storage from SHF-N_HEAT
                       ds_N_HEAT_diff.rename('MHTD'), # Meridional Heat Transport Difference
                       ds_sa_SHF.rename('SHF')]) # Total Surface Heat Flux
ds_out_var.attrs['description'] = 'Heat balance components for the South Atlantic from PI Control'
ds_out_var.attrs['units'] = 'PW'
ds_out_var.attrs['author'] = 'Mauricio Rocha'
ds_out_var.attrs['email'] = 'mauricio.rocha@usp.br'
# create a directory on scratch to save the output
path = '/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/'.format(getpass.getuser())
os.system('mkdir -p '+path)
ds_out_var.to_netcdf(path+'heat_balance_components_PI_control.nc')
del ds_out_var

### Calculate the heat stored per layer

In [ ]:
%%time
warnings.simplefilter("ignore")
layers=('0','1e+5','6e+5')
for layer in range(0,len(layers)-1):
    st=f'ds_HS_TEMP=ds_sa_TEND_TEMP.sel(z_t=slice({layers[layer]},{layers[layer+1]}))*dz.sel(z_t=slice({layers[layer]},{layers[layer+1]}))' # 1- Multiply by dz. Unit: oC.s-1.m
    exec(st); del st
    st=f'ds_HS_TEMP=ds_HS_TEMP*ds_sa_TAREA.sel(z_t=slice({layers[layer]},{layers[layer+1]}))' # 2- Multiply by the area. Unit: oC.s-1.m3
    exec(st); del st
    ds_HS_TEMP=ds_HS_TEMP.sum(dim=['z_t','nlon','nlat']) # 3- Integral in dz,dy,dx. Unit: oC.s-1.m3
    ds_HS_TEMP=ds_HS_TEMP*1026 # 4- Multiply by the density of the sea water. Unit: oC.s-1.kg
    ds_HS_TEMP=ds_HS_TEMP*3996 # 5- Multiply by the heat capacity of the sea water. Unit: W
    ds_HS_TEMP=ds_HS_TEMP*1e-15 # 6- Get the variable in PW 
    st=f'ds_HS_TEMP_merged_{layer}=ds_HS_TEMP.load()'
    exec(st); del st
del ds_HS_TEMP, ds_sa_TAREA, ds_HS_TEMP, ds_sa_TEND_TEMP

In [ ]:
ds_HS_TEMP_merged_0.resample(time='1Y', closed='left').mean('time').plot(label='0-1000')
ds_HS_TEMP_merged_1.resample(time='1Y', closed='left').mean('time').plot(label='1000-6000')
plt.legend()
plt.show()

In [ ]:
ds_out_var = xr.merge([ds_HS_TEMP_merged_0.rename('HS_0'), # Heat Storage (first layer)
                       ds_HS_TEMP_merged_1.rename('HS_1'), # Heat Storage (second layer)
                      ]) # Total Surface Heat Flux
ds_out_var.attrs['description'] = 'Heat balance components from the PI Control per layers for the South Atlantic: HS_0 (0-1000m), HS_1 (1000-6000m)'
ds_out_var.attrs['units'] = 'PW'
ds_out_var.attrs['author'] = 'Mauricio Rocha'
ds_out_var.attrs['email'] = 'mauricio.rocha@usp.br'
# create a directory on scratch to save the output
path = '/glade/scratch/mauricio/Data/LENS2/HEAT_BALANCE/'.format(getpass.getuser())
os.system('mkdir -p '+path)
ds_out_var.to_netcdf(path+'heat_storage_per_layer_0_6000m_PI_control.nc')